In [1]:
import geojson
import pandas as pd
import json

In [2]:
# import folium

# map_2 = folium.Map(location=[39.2904, -76.6122],
#                    zoom_start=11)
# for s in parking_citations_data_withgeo["GeoPoint"].values[0:60]:
#     try:
#         s1 = s[1:len(s)-1].split(",")
#         latitude = float(s1[0].strip())
#         longitude = float(s1[1].strip())
#         folium.CircleMarker([latitude, longitude],
#                     radius=1,
#                     color='#3186cc',
#                     fill_color='#3186cc',
#                    ).add_to(map_2)
#     except:
#         print(s1)
# map_2

In [3]:
baltimore_city_geojson_string = open("data/Baltimore City Line.geojson").read()
baltimore_city_geojson = geojson.loads(baltimore_city_geojson_string)

minLat = float("inf")
maxLat = float("-inf")
minLong = float("inf")
maxLong = float("-inf")

#Longitude, Latitude
for coordinate in baltimore_city_geojson["features"][0]["geometry"]["coordinates"][0]:
    lat = coordinate[1]
    if(lat < minLat):
        minLat = lat
    if(lat > maxLat):
        maxLat = lat
    long = coordinate[0]
    if(long < minLong):
        minLong = long
    if(long > maxLong):
        maxLong = long

baltimore_geo_extremes = {
    "minLatitude" : minLat,
    "maxLatitude" : maxLat,
    "minLongitude" : minLong,
    "maxLongitude" : maxLong
}

with open('modified_data/geo_extremes.json', 'w') as outfile:
    json.dump(baltimore_geo_extremes, outfile)